In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
from tensorflow.examples.tutorials.mnist import input_data

In [6]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [7]:
type(mnist)

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [8]:
mnist.train.num_examples

55000

In [9]:
mnist.test.num_examples

10000

In [10]:
def initalize_weights(filter_shape):
    init_random_dist=tf.truncated_normal(filter_shape,stddev=0.1)
    return(tf.Variable(init_random_dist))

In [31]:
def initialize_bias(bias_shape):
    initail_bias_vals=tf.constant(0.1,shape=bias_shape)
    return(tf.Variable(initail_bias_vals))

In [25]:
def create_convolution_layer_and_compute_dot_products(inputs,filter_shape):
    filter_intialized_with_weights=initalize_weights(filter_shape)
    conv_layer_outputs=tf.nn.conv2d(inputs,filter_intialized_with_weights,strides=[1,1,1,1],padding='SAME')
    return (conv_layer_outputs)

In [33]:
def create_relu_layer_and_compute_dotproduct_plus_b(inputs,filter_shape):
    b=initialize_bias([filter_shape[3]])
    relu_layer_outputs=tf.nn.relu(inputs + b)
    return(relu_layer_outputs)

In [37]:
def create_maxpool2by2_and_reduce_spatial_size(inputs):
    pooling_layer_outputs=tf.nn.max_pool(inputs,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
    return (pooling_layer_outputs)

In [44]:
def create_fully_connected_layer_and_compute_dotproduct_plus_bias(inputs,output_size):
    input_size=int(inputs.get_shape()[1])
    W=initalize_weights([input_size,output_size])
    b=initialize_bias([output_size])
    fc_xW_plus_bias_outputs=tf.matmul(inputs,W)+b
    return (fc_xW_plus_bias_outputs)
    

In [16]:
x = tf.placeholder(tf.float32,shape=[None,784])


In [17]:
y_true=tf.placeholder(tf.float32,[None,10])

In [19]:
x_image=tf.reshape(x,[-1,28,28,1])

Create 1st Convolutional Layer,ReLu Layer and perform computation: x*W+b

In [35]:
conv_layer_1_outputs = create_convolution_layer_and_compute_dot_products(x_image,filter_shape=[5,5,1,32])
conv_relu_layer_1_outputs \
    = create_relu_layer_and_compute_dotproduct_plus_b(conv_layer_1_outputs,filter_shape=[5,5,1,32])

## Create 1st Pooling Layer and Reduce Spatial Size

In [38]:
pooling_layer_1_outputs = create_maxpool2by2_and_reduce_spatial_size(conv_relu_layer_1_outputs)

## Create 1st Convolutional Layer,ReLu Layer and perform computation: x*W+b

In [40]:
conv_layer_2_outputs = create_convolution_layer_and_compute_dot_products(pooling_layer_1_outputs,filter_shape=[5,5,32,64])
                
conv_relu_layer_2_outputs \
    = create_relu_layer_and_compute_dotproduct_plus_b(conv_layer_2_outputs,filter_shape=[5,5,32,64])

### Create 1st Pooling Layer and Reduce Spatial Size

In [41]:
pooling_layer_2_outputs = create_maxpool2by2_and_reduce_spatial_size(conv_relu_layer_2_outputs)

### Reshape/flatten data making it ready to be fed into 1st FC layer

In [42]:
pooling_layer_2_outputs_flat=tf.reshape(pooling_layer_2_outputs,[-1,7*7*64])


### create Droupout layer and dropout a fraction of outputs randomly

In [45]:
fc_layer_1_outputs \
    =create_fully_connected_layer_and_compute_dotproduct_plus_bias(pooling_layer_2_outputs_flat,output_size=1024)
fc_relu_layer_1_outputs=tf.nn.relu(fc_layer_1_outputs)

### create dropout layer and dropout a fraction of outputs randomly

In [46]:
hold_prob = tf.placeholder(tf.float32)
fc_dropout_outputs=tf.nn.dropout(fc_layer_1_outputs,keep_prob=hold_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Create Final FC layer compute(x.W +b), and produce Final Outputs

In [47]:
y_pred=create_fully_connected_layer_and_compute_dotproduct_plus_bias(fc_dropout_outputs,output_size=10)

### Define Loss Function and Calculate Softmax Cross Entropy Loss

In [48]:
softmax_cross_entropy_loss=tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred)
cross_entropy_mean=tf.reduce_mean(softmax_cross_entropy_loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### Create an Optimizer to Optimize CNN model and set learning rate

In [50]:
optimizer=tf.train.AdamOptimizer(learning_rate=0.001)

### Create a Trainer to Ttaining CNN Model

In [51]:
cnn_trainer=optimizer.minimize(cross_entropy_mean)

### Create a Variable initializer to initaialize ALL variables

In [52]:
vars_initializer=tf.global_variables_initializer()

### Set the Steps

In [53]:
steps =5000

### Run tf.session() to Train and Test Deep learning CNN model

In [56]:
with tf.Session() as sess:
    sess.run(vars_initializer)
    
    for i in range(steps):
        batch_x,batch_y=mnist.train.next_batch(50)
        sess.run(cnn_trainer,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        if i%100 == 0:
            print('On STEP : {}'.format(i))
            print('Accuracy :')
            
            matches =tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc =tf.reduce_mean(tf.cast(matches,tf.float32))
            test_accuracy = sess.run( acc, feed_dict={x:mnist.test.images,\
                                                     y_true:mnist.test.labels,\
                                                     hold_prob:1.0})
            print(test_accuracy)
            print('\n')
    

On STEP : 0
Accuracy :
0.1204


On STEP : 100
Accuracy :
0.9323


On STEP : 200
Accuracy :
0.9576


On STEP : 300
Accuracy :
0.966


On STEP : 400
Accuracy :
0.9682


On STEP : 500
Accuracy :
0.9728


On STEP : 600
Accuracy :
0.9746


On STEP : 700
Accuracy :
0.9787


On STEP : 800
Accuracy :
0.9756


On STEP : 900
Accuracy :
0.974


On STEP : 1000
Accuracy :
0.9792


On STEP : 1100
Accuracy :
0.9767


On STEP : 1200
Accuracy :
0.9795


On STEP : 1300
Accuracy :
0.9761


On STEP : 1400
Accuracy :
0.9805


On STEP : 1500
Accuracy :
0.9823


On STEP : 1600
Accuracy :
0.9753


On STEP : 1700
Accuracy :
0.9829


On STEP : 1800
Accuracy :
0.981


On STEP : 1900
Accuracy :
0.9811


On STEP : 2000
Accuracy :
0.9841


On STEP : 2100
Accuracy :
0.9818


On STEP : 2200
Accuracy :
0.9827


On STEP : 2300
Accuracy :
0.9837


On STEP : 2400
Accuracy :
0.9864


On STEP : 2500
Accuracy :
0.9837


On STEP : 2600
Accuracy :
0.9846


On STEP : 2700
Accuracy :
0.9854


On STEP : 2800
Accuracy :
0.9866


